In [1]:
import numpy as np
import pandas as pd
import requests
import pickle

In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Get distance and time matrices using OSRM API

## API

In [3]:
stops_df = pd.read_excel('../Data collection_v3.xlsx',sheet_name='Stops 2.Tour (Mo-Fr)')

In [4]:
stops_df

,No,Customer ID,Stop name,Duration (in min),Dependency,Type,Stop type,Address,Postal Code,Town,Opening,Time from,Time to,"Latitude, Longitude"
0,1,9012,Fiwa Nord (9012),00:15:00,NaN,Delivery,Store,Kirchhainer Str. 12,3238,Finsterwalde,NaN,NaN,NaN,"51.63647133321395, 13.704593369766636"
1,2,9005,Fiwa Markt (9005),00:15:00,NaN,Delivery,Store,Markt 28,3238,Finsterwalde,NaN,NaN,NaN,"51.629624536601284, 13.708361310244655"
2,3,9002,Fiwa Süd (9002),00:15:00,NaN,Delivery,Store,Rosa-Luxemburg-Straße 69,3238,Finsterwalde,NaN,NaN,NaN,"51.62273511871606, 13.715053667916017"
3,4,9003,Fiwa Edeka (9003),00:15:00,NaN,Delivery,Store,Berliner Str. 20 b,3238,Finsterwalde,NaN,NaN,NaN,"51.634787798008055, 13.706825712095014"
4,5,9017,Calau (9017),00:15:00,NaN,Delivery,Store,Cottbuser Str. 44,3205,Calau,NaN,NaN,NaN,"51.746938845168074, 13.950017440934058"
5,6,9010,Lübbenau (9010),00:15:00,NaN,Delivery,Store,Ehm-Welk-Str. 35,3222,Lübbenau/Spreewald,NaN,NaN,NaN,"51.868367097476884, 13.968177200459339"
6,7,9025,Cottbus (9025),00:15:00,NaN,Delivery,Store,Rosenstraße 4,3046,Cottbus,NaN,NaN,NaN,"51.75854401147092, 14.337405602306243"
7,8,9018,Kirchhain Markt (9018),00:15:00,NaN,Delivery,Store,Am Markt 3,3253,Doberlug-Kirchhain,NaN,NaN,NaN,"51.63745434921447, 13.562850371127455"
8,9,9026,Kirchhain Edeke (9026),00:15:00,NaN,Delivery,Store,Gerberstraße 22,3253,Doberlug-Kirchhain,NaN,NaN,NaN,"51.63772004525862, 13.565081968065178"
9,10,9007,Herzberg Penny (9007),00:15:00,NaN,Delivery,Store,Grochwitzer Str. 2,4916,Herzberg (Elster),NaN,NaN,NaN,"51.69132509071261, 13.226475610246515"


In [5]:
len(stops_df['Customer ID']) == len(stops_df['Customer ID'].unique())

True

In [6]:
no_to_customer_id_dict = {
    **{0: 'depot'},
    **pd.Series(stops_df['Customer ID'].values,index=stops_df['No']).to_dict()
}

In [7]:
no_to_customer_id_dict

{0: 'depot',
 1: 9012,
 2: 9005,
 3: 9002,
 4: 9003,
 5: 9017,
 6: 9010,
 7: 9025,
 8: 9018,
 9: 9026,
 10: 9007,
 11: 9023,
 12: 9006,
 13: 9019,
 14: 9024,
 15: 9011,
 16: 9014,
 17: 9009,
 18: 9022,
 19: 9031,
 20: 9015,
 21: 9008}

In [9]:
base_url = 'http://router.project-osrm.org/table/v1/driving/'

In [55]:
# The format for OSRM is longitude/latitude, not latitude/longitude
long_lat_str = ';'.join(
    [
        ','.join(x.replace(' ','').split(',')[::-1]) for x in list(stops_df['Latitude, Longitude'])
    ])

# Add the coordinates of the production centre at Gewerbegebiet Südstraße 5
long_lat_str = '13.5853717,51.6286970;'+long_lat_str

In [56]:
long_lat_str

'13.5853717,51.6286970;13.704593369766636,51.63647133321395;13.708361310244655,51.629624536601284;13.715053667916017,51.62273511871606;13.706825712095014,51.634787798008055;13.950017440934058,51.746938845168074;13.968177200459339,51.868367097476884;14.337405602306243,51.75854401147092;13.562850371127455,51.63745434921447;13.565081968065178,51.63772004525862;13.226475610246515,51.69132509071261;13.235057625589485,51.692670038544875;13.646774398604009,51.695714482922284;13.713940685115906,51.85297305510116;13.895785640939774,51.94173240024925;13.235616371615365,51.58780069552828;13.394921112091476,51.51596315711454;13.533434312090055,51.465021495479824;13.870686656268836,51.467827636403335;13.999790271460139,51.559313325646336;14.012146598600864,51.58804290089093;13.255818242407956,51.588042199301995'

In [57]:
params = {'annotations': 'distance,duration'}

In [58]:
# https://project-osrm.org/docs/v5.22.0/api/#general-options
#response = (
    requests.get(base_url+long_lat_str,params=params)
)

In [60]:
#with open('osrm_response_20221121.pickle', 'wb') as handle:
    pickle.dump(response, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Unpickle

In [8]:
with open('osrm_response_20221121.pickle', 'rb') as handle:
    response = pickle.load(handle)

In [9]:
response.json()

{'code': 'Ok',
 'distances': [[0,
   9050.6,
   10003.6,
   11169.6,
   9462.9,
   36421,
   54742.5,
   85087.1,
   2875.1,
   2975.1,
   31026.7,
   30210.7,
   10038,
   31882.1,
   48765.2,
   29193.7,
   23756.4,
   36230.9,
   42751.1,
   34553.5,
   33043.4,
   27690.7],
  [9051.4,
   0,
   953,
   2010.6,
   412.3,
   27370.4,
   45691.9,
   76036.6,
   11391,
   11491.1,
   39542.6,
   38726.7,
   9071.8,
   30915.8,
   47798.9,
   37709.7,
   32272.3,
   28501.1,
   33700.5,
   25502.9,
   23992.8,
   36206.6],
  [10198.9,
   1215,
   0,
   1057.6,
   935.2,
   27215.9,
   45537.3,
   75882,
   12538.5,
   12638.6,
   40690.1,
   39874.2,
   10065,
   31909,
   48792.1,
   38611.7,
   33174.3,
   27022,
   33546,
   25348.4,
   23838.3,
   37108.7],
  [11174.5,
   1959.5,
   1073.6,
   0,
   1723.1,
   27933,
   46254.5,
   76599.2,
   14810.3,
   14910.4,
   42961.9,
   42146,
   10809.4,
   32653.5,
   59430.2,
   39514.4,
   36736.3,
   25964.5,
   34263.1,
   26065.5,
   

In [10]:
# Distance and time matrices
distances = response.json()['distances']
durations = response.json()['durations']

# Optimise on distances using OR-Tools

In [11]:
def create_data_model(num_vehicles):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distances
    data['num_vehicles'] = num_vehicles
    data['depot'] = 0
    return data

In [12]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    sum_distances = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f' {manager.IndexToNode(index)} ({no_to_customer_id_dict[manager.IndexToNode(index)]}) -> '
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += f'{manager.IndexToNode(index)} ({no_to_customer_id_dict[manager.IndexToNode(index)]})\n'
        plan_output += 'Distance of the route: {}km\n'.format(route_distance/1000)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
        sum_distances += route_distance
    print('Maximum of the route distances: {}km'.format(max_route_distance/1000))
    print(f'Sum of the route distances: {sum_distances/1000}km')

In [13]:
def solve_distances(
    num_vehicles,
    vehicle_max_dist, # in km
    distance_global_span_cost_coefficient):
    
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(num_vehicles)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        vehicle_max_dist*1000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(distance_global_span_cost_coefficient)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')

In [44]:
solve_distances(1,1000,100)

Objective: 28988919
Route for vehicle 0:
 0 (depot) ->  1 (9012) ->  4 (9003) ->  2 (9005) ->  3 (9002) ->  12 (9006) ->  13 (9019) ->  14 (9024) ->  6 (9010) ->  5 (9017) ->  7 (9025) ->  20 (9015) ->  19 (9031) ->  18 (9022) ->  17 (9009) ->  16 (9014) ->  21 (9008) ->  15 (9011) ->  10 (9007) ->  11 (9023) ->  8 (9018) ->  9 (9026) -> 0 (depot)
Distance of the route: 287.019km

Maximum of the route distances: 287.019km
Sum of the route distances: 287.019km


In [45]:
solve_distances(2,1000,100)

Objective: 17284316
Route for vehicle 0:
 0 (depot) ->  1 (9012) ->  5 (9017) ->  7 (9025) ->  6 (9010) ->  14 (9024) ->  13 (9019) ->  12 (9006) -> 0 (depot)
Distance of the route: 169.514km

Route for vehicle 1:
 0 (depot) ->  4 (9003) ->  2 (9005) ->  3 (9002) ->  20 (9015) ->  19 (9031) ->  18 (9022) ->  17 (9009) ->  16 (9014) ->  21 (9008) ->  15 (9011) ->  10 (9007) ->  11 (9023) ->  8 (9018) ->  9 (9026) -> 0 (depot)
Distance of the route: 163.402km

Maximum of the route distances: 169.514km
Sum of the route distances: 332.916km


In [46]:
solve_distances(3,1000,100)

Objective: 15689518
Route for vehicle 0:
 0 (depot) ->  4 (9003) ->  2 (9005) ->  3 (9002) ->  5 (9017) ->  19 (9031) ->  18 (9022) ->  17 (9009) ->  16 (9014) ->  8 (9018) ->  9 (9026) -> 0 (depot)
Distance of the route: 152.298km

Route for vehicle 1:
 0 (depot) ->  14 (9024) ->  13 (9019) ->  11 (9023) ->  10 (9007) ->  15 (9011) ->  21 (9008) -> 0 (depot)
Distance of the route: 150.072km

Route for vehicle 2:
 0 (depot) ->  1 (9012) ->  20 (9015) ->  7 (9025) ->  6 (9010) ->  12 (9006) -> 0 (depot)
Distance of the route: 152.348km

Maximum of the route distances: 152.348km
Sum of the route distances: 454.718km


In [47]:
solve_distances(4,1000,100)

Objective: 14328744
Route for vehicle 0:
 0 (depot) ->  4 (9003) ->  2 (9005) ->  3 (9002) ->  19 (9031) ->  18 (9022) ->  17 (9009) ->  16 (9014) -> 0 (depot)
Distance of the route: 119.319km

Route for vehicle 1:
 0 (depot) ->  9 (9026) ->  8 (9018) ->  11 (9023) ->  10 (9007) ->  15 (9011) ->  21 (9008) -> 0 (depot)
Distance of the route: 72.868km

Route for vehicle 2:
 0 (depot) ->  13 (9019) ->  14 (9024) ->  6 (9010) ->  12 (9006) -> 0 (depot)
Distance of the route: 109.677km

Route for vehicle 3:
 0 (depot) ->  20 (9015) ->  7 (9025) ->  5 (9017) ->  1 (9012) -> 0 (depot)
Distance of the route: 138.88km

Maximum of the route distances: 138.88km
Sum of the route distances: 440.744km


In [48]:
solve_distances(5,1000,100)

Objective: 14328744
Route for vehicle 0:
 0 (depot) ->  4 (9003) ->  2 (9005) ->  3 (9002) ->  19 (9031) ->  18 (9022) ->  17 (9009) ->  16 (9014) -> 0 (depot)
Distance of the route: 119.319km

Route for vehicle 1:
 0 (depot) ->  9 (9026) ->  8 (9018) ->  11 (9023) ->  10 (9007) ->  15 (9011) ->  21 (9008) -> 0 (depot)
Distance of the route: 72.868km

Route for vehicle 2:
 0 (depot) ->  13 (9019) ->  14 (9024) ->  6 (9010) ->  12 (9006) -> 0 (depot)
Distance of the route: 109.677km

Route for vehicle 3:
 0 (depot) -> 0 (depot)
Distance of the route: 0.0km

Route for vehicle 4:
 0 (depot) ->  20 (9015) ->  7 (9025) ->  5 (9017) ->  1 (9012) -> 0 (depot)
Distance of the route: 138.88km

Maximum of the route distances: 138.88km
Sum of the route distances: 440.744km


In [49]:
solve_distances(6,1000,100)

Objective: 14328744
Route for vehicle 0:
 0 (depot) ->  4 (9003) ->  2 (9005) ->  3 (9002) ->  19 (9031) ->  18 (9022) ->  17 (9009) ->  16 (9014) -> 0 (depot)
Distance of the route: 119.319km

Route for vehicle 1:
 0 (depot) ->  9 (9026) ->  8 (9018) ->  11 (9023) ->  10 (9007) ->  15 (9011) ->  21 (9008) -> 0 (depot)
Distance of the route: 72.868km

Route for vehicle 2:
 0 (depot) ->  13 (9019) ->  14 (9024) ->  6 (9010) ->  12 (9006) -> 0 (depot)
Distance of the route: 109.677km

Route for vehicle 3:
 0 (depot) -> 0 (depot)
Distance of the route: 0.0km

Route for vehicle 4:
 0 (depot) -> 0 (depot)
Distance of the route: 0.0km

Route for vehicle 5:
 0 (depot) ->  20 (9015) ->  7 (9025) ->  5 (9017) ->  1 (9012) -> 0 (depot)
Distance of the route: 138.88km

Maximum of the route distances: 138.88km
Sum of the route distances: 440.744km


In [50]:
solve_distances(7,1000,100)

Objective: 14328744
Route for vehicle 0:
 0 (depot) ->  4 (9003) ->  2 (9005) ->  3 (9002) ->  19 (9031) ->  18 (9022) ->  17 (9009) ->  16 (9014) -> 0 (depot)
Distance of the route: 119.319km

Route for vehicle 1:
 0 (depot) ->  9 (9026) ->  8 (9018) ->  11 (9023) ->  10 (9007) ->  15 (9011) ->  21 (9008) -> 0 (depot)
Distance of the route: 72.868km

Route for vehicle 2:
 0 (depot) ->  13 (9019) ->  14 (9024) ->  6 (9010) ->  12 (9006) -> 0 (depot)
Distance of the route: 109.677km

Route for vehicle 3:
 0 (depot) -> 0 (depot)
Distance of the route: 0.0km

Route for vehicle 4:
 0 (depot) -> 0 (depot)
Distance of the route: 0.0km

Route for vehicle 5:
 0 (depot) -> 0 (depot)
Distance of the route: 0.0km

Route for vehicle 6:
 0 (depot) ->  20 (9015) ->  7 (9025) ->  5 (9017) ->  1 (9012) -> 0 (depot)
Distance of the route: 138.88km

Maximum of the route distances: 138.88km
Sum of the route distances: 440.744km
